In [2]:
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import regex as re
import requests
import time

pd.set_option('display.max_rows', None)
pd.set_option('mode.chained_assignment',None)

In [3]:
headers = {
   'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
   'Accept-Encoding' : 'gzip, deflate, br',
   'Accept-Language' : 'en-US,en;q=0.5',
   'cache-control' : 'no-cache',
   'content-encoding' : 'gzip',
   'Cookie' : 'x-amz-continuous-deployment-state=AYABeNfmFSgaeHo%2FLtR+c8h+1+QAPgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADFr4P9HOJU4H5lQbnQAw7AEiGQ0HM1Fk4U7zntBSGv2xUhbmywBg40MJxKPDYY62t9qinQgjNGOrqkxhwHasAgAAAAAMAAQAAAAAAAAAAAAAAAAAAMafJVan5wcqx9P0nZC5Yc7%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAwxkuqjvKJA7NJe6hGuniFPlxg9rVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3p; search=6|1708819459747%7…AAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADFr4P9HOJU4H5lQbnQAw7AEiGQ0HM1Fk4U7zntBSGv2xUhbmywBg40MJxKPDYY62t9qinQgjNGOrqkxhwHasAgAAAAAMAAQAAAAAAAAAAAAAAAAAAMafJVan5wcqx9P0nZC5Yc7%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAwxkuqjvKJA7NJe6hGuniFPlxg9rVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3prVuNKZ4nFi3p; JSESSIONID=F41F9EA33C943545A252BC4A41C32C2D',
   'Sec-Fetch-Dest' : 'document',
   'Sec-Fetch-Mode' : 'navigate',
   'Sec-Fetch-Site' : 'same-origin',
   'Sec-Fetch-User' : '?1',
   'Upgrade-Insecure-Requests' : '1',
   'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; rv:109.0) Gecko/20100101 Firefox/116.0'

}

In [16]:
# get around the response outputting JSON and getting an error from structure, manually build a new response list
# write loops to create lists for each page request 
# useful for scraping through more than a single page


with requests.session() as s:
    city = 'santa-barbara-county-ca/'
    page = 1
    end_page = 10
    url = ''
    url_list = []
    
    while page <= end_page:
        url = 'https://www.zillow.com/' +city+'/rentals/' f'{page}_p/'
        url_list.append(url)
        page += 1
    
    request = ''
    request_list = []
    
    for url in url_list:
        #time.sleep(2)
        request = s.get(url, headers=headers)
        print(url)
        request_list.append(request)
    
soup = ''
soup_list = []

# loops through the request list in order to fill the BeautifulSoup list
for request in request_list:
    soup = BeautifulSoup(request.content, 'html.parser')
    soup_list.append(soup)

https://www.zillow.com/santa-barbara-county-ca//rentals/1_p/
https://www.zillow.com/santa-barbara-county-ca//rentals/2_p/
https://www.zillow.com/santa-barbara-county-ca//rentals/3_p/
https://www.zillow.com/santa-barbara-county-ca//rentals/4_p/
https://www.zillow.com/santa-barbara-county-ca//rentals/5_p/
https://www.zillow.com/santa-barbara-county-ca//rentals/6_p/
https://www.zillow.com/santa-barbara-county-ca//rentals/7_p/
https://www.zillow.com/santa-barbara-county-ca//rentals/8_p/
https://www.zillow.com/santa-barbara-county-ca//rentals/9_p/
https://www.zillow.com/santa-barbara-county-ca//rentals/10_p/


In [5]:
# create empty lists of features per property card
df_list = [] # list of dataframes (each data frame contains properties from a page)
beds = []
price = []
address = []

for soup in soup_list:
    df = pd.DataFrame()
    for i in soup:
        try:
            address_element = soup.find(class_= 'StyledPropertyCardDataArea-c11n-8-84-3__sc-yipmu-0 jnnxAW property-card-link')
        except AttributeError:
            address_element = None
        address.append(address_element)
        try:
            price_element = soup.find(class_='PropertyCardWrapper__StyledPriceGridContainer-srp__sc-16e8gqd-0 kSsByo')
        except AttributeError:
            price_element = None
        price.append(price_element)
        try:
            beds_element = soup.find("ul", class_='StyledPropertyCardHomeDetailsList-c11n-8-84-3__sc-1xvdaej-0 eYPFID')
        except AttributeError:
            beds_element = None
        beds.append(beds_element)
        
    df['price'] = price
    df['address'] = address
    df['beds'] = beds

    df_list.append(df)
    

In [24]:
# create empty lists of features per property card
df_list = [] # list of dataframes (each data frame contains properties from a page)
beds = []
price = []
address = []


address_element = soup.find(class_= 'StyledPropertyCardDataArea-c11n-8-84-3__sc-yipmu-0 jnnxAW property-card-link')


df1 = pd.DataFrame()
df1["address"] = address_element
#for soup in soup_list:
#    df = pd.DataFrame()
   # for i in soup:
    #    try:
     #       address_element = soup.find(class_= 'StyledPropertyCardDataArea-c11n-8-84-3__sc-yipmu-0 jnnxAW property-card-link')
      #  except AttributeError:
       #     address_element = None
        #address.append(address_element)
        #try:
         #   price_element = soup.find(class_='PropertyCardWrapper__StyledPriceGridContainer-srp__sc-16e8gqd-0 kSsByo')
        #except AttributeError:
         #   price_element = None
        #price.append(price_element)
        #try:
         #   beds_element = soup.find("ul", class_='StyledPropertyCardHomeDetailsList-c11n-8-84-3__sc-1xvdaej-0 eYPFID')
        #except AttributeError:
        #    beds_element = None
        #beds.append(beds_element)
        
    #df['price'] = price
    #df['address'] = address
    #df['beds'] = beds

    #df_list.append(df)
    

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [15]:
df_list[9]



,price,address,beds
0,"[[$4,220+ 1 bd]]","[[Arrive Los Carneros II | 6720 Calle Koral, G...","[[[$4,795+], , , 2 bds], [[$5,400+], , , 3..."
1,"[[$4,220+ 1 bd]]","[[Arrive Los Carneros II | 6720 Calle Koral, G...","[[[$4,795+], , , 2 bds], [[$5,400+], , , 3..."
2,"[[$4,220+ 1 bd]]","[[Arrive Los Carneros II | 6720 Calle Koral, G...","[[[$4,795+], , , 2 bds], [[$5,400+], , , 3..."
3,"[[$2,900/mo]]","[[(undisclosed Address), Santa Barbara, CA 931...","[[[1], , [bd]], [[1], , [ba]], [[750], , [s..."
4,"[[$2,900/mo]]","[[(undisclosed Address), Santa Barbara, CA 931...","[[[1], , [bd]], [[1], , [ba]], [[750], , [s..."
5,"[[$2,900/mo]]","[[(undisclosed Address), Santa Barbara, CA 931...","[[[1], , [bd]], [[1], , [ba]], [[750], , [s..."
6,"[[$5,850/mo]]","[[1055 Toro Canyon Rd, Santa Barbara, CA 93108]]","[[[3], , [bds]], [[2], , [ba]], [[1,725], ,..."
7,"[[$5,850/mo]]","[[1055 Toro Canyon Rd, Santa Barbara, CA 93108]]","[[[3], , [bds]], [[2], , [ba]], [[1,725], ,..."
8,"[[$5,850/mo]]","[[1055 Toro Canyon Rd, Santa Barbara, CA 93108]]","[[[3], , [bds]], [[2], , [ba]], [[1,725], ,..."
9,"[[$18,000/mo]]","[[762 Westmont Rd, Santa Barbara, CA 93108]]","[[[3], , [bds]], [[3.5], , [ba]], [[3,510], ..."
